In [1]:
from keras.models import Sequential
from keras.layers import LSTM,Dropout
from keras.layers import Dense
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import decomposition
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import scikitplot as skplt
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from scipy import interp
from itertools import cycle
import time
from keras import metrics
from keras import optimizers
from keras import initializers
from keras.callbacks import ModelCheckpoint
import pprint
from keras.callbacks import Callback

Using TensorFlow backend.


In [2]:
kdd_dataset = pd.read_csv('KDD.preProcessed.csv')
kdd_dataset.head()
#kdd_dataset.dropna()
np.random.seed(42)

In [3]:
columns = list(kdd_dataset)
features = columns
features.remove('label_DoS')
features.remove('label_Normal')
features.remove('label_Probe')
features.remove('label_R2L')
features.remove('label_U2R')

In [4]:

DoS = kdd_dataset.loc[kdd_dataset['label_DoS'] == 1]
Normal = kdd_dataset.loc[kdd_dataset['label_Normal'] == 1]
Probe = kdd_dataset.loc[kdd_dataset['label_Probe'] == 1]
R2L = kdd_dataset.loc[kdd_dataset['label_R2L'] == 1]
U2R = kdd_dataset.loc[kdd_dataset['label_U2R'] == 1]


In [5]:
DoS = DoS.sample(n=1000,random_state = 42)
Normal = Normal.sample(n=1000, random_state = 42)
Probe = Probe.sample(n=1000, random_state=42)
R2L = R2L.sample(n=1126, random_state=42)
'''
DoS = DoS.sample(n=100000,random_state = 42)
Normal = Normal.sample(n=100000, random_state = 42)
Probe = Probe.sample(n=40000, random_state=42)
R2L = R2L.sample(n=1126, random_state=42)
'''

'\nDoS = DoS.sample(n=100000,random_state = 42)\nNormal = Normal.sample(n=100000, random_state = 42)\nProbe = Probe.sample(n=40000, random_state=42)\nR2L = R2L.sample(n=1126, random_state=42)\n'

In [87]:
#U2R = U2R.append([U2R]*500,ignore_index=True)
#R2L = R2L.append([R2L]*100,ignore_index=True)
#DoS

In [6]:
reduced_dataset = pd.concat([DoS,Normal,Probe,R2L,U2R])
kdd_dataset = reduced_dataset.sample(n=len(reduced_dataset), random_state = 42)

In [88]:

x = kdd_dataset[features].values
y = kdd_dataset.iloc[:,53:].values
print("number of datapoints = {} and number of features = {}".format(len(x),len(x[0])))

number of datapoints = 4148 and number of features = 53


In [89]:
# Counting occurrences
from collections import Counter
print(Counter([tuple(x) for x in y]))

Counter({(0.0, 0.0, 0.0, 1.0, 0.0): 1126, (1.0, 0.0, 0.0, 0.0, 0.0): 1000, (0.0, 1.0, 0.0, 0.0, 0.0): 1000, (0.0, 0.0, 1.0, 0.0, 0.0): 1000, (0.0, 0.0, 0.0, 0.0, 1.0): 22})


In [90]:
#PCA

print("Shape before transformation - {}".format(np.asarray(x).shape))
pca = decomposition.PCA(n_components=10)
pca.fit(x)
x_pca = pca.transform(x)
print("Shape after transformation - {}".format(x_pca.shape))
x_pca = x_pca.tolist()



Shape before transformation - (4148, 53)
Shape after transformation - (4148, 10)


In [91]:
from sklearn.model_selection import train_test_split
X_tr, X_val, Y_tr, Y_val = train_test_split(x_pca, y,
                                                    stratify=y, 
                                                    test_size=0.2,
                                                    random_state=42)


In [92]:
X_train, X_test, Y_train, Y_test = train_test_split(X_tr, Y_tr,
                                                    stratify=Y_tr, 
                                                    test_size=0.25,
                                                    random_state=42)
max_len = len(X_train)

In [93]:
def processTrainTestArrays(x,y):
    x=np.asarray(x)
    #y=np.asarray(y)
    x = np.reshape(x, (x.shape[0], x.shape[1],1))
    y = [np.argmax(item) for item in y]
    y = np.asarray(y)
    y = np.reshape(y, (y.shape[0], 1))


    return x,y

In [94]:
def onehotencode(y):
    from keras.utils.np_utils import to_categorical
    y_binary = to_categorical(y)
    return y_binary

In [95]:
# define model

def create_model(X_train,Y_train, number_of_units):
    Batch_size = 1
    randomInit = initializers.RandomUniform(seed=42)
    nadam = optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
    
    model = Sequential()
    #model.add(LSTM(number_of_units, kernel_initializer = randomInit, bias_initializer = randomInit, stateful = True,
    #               batch_input_shape=(Batch_size,X_train.shape[1], X_train.shape[2]),return_sequences=True))
    model.add(LSTM(number_of_units, kernel_initializer = randomInit, bias_initializer = randomInit, stateful = True,
                   batch_input_shape=(Batch_size,1,1),return_sequences=False))
    
    #model.add(Dropout(0.1))
    #model.add(LSTM(number_of_units, kernel_initializer = randomInit, bias_initializer = randomInit, stateful = True,
    #               batch_input_shape=(Batch_size,X_train.shape[1], X_train.shape[2]),return_sequences=True))
    #model.add(Dropout(0.1))
    #model.add(LSTM(number_of_units, kernel_initializer = randomInit, bias_initializer = randomInit, stateful = True,
    #               batch_input_shape=(Batch_size,X_train.shape[1], X_train.shape[2])))
    #model.add(Dropout(0.1))
    model.add(Dense(Y_train.shape[1], activation='softmax'))
    
    nadam = optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
    model.compile(loss='mean_squared_error', optimizer=nadam, metrics=['accuracy'])

    print(model.summary())
    return model

In [97]:
def compute_metrics(predicted, Y_test):
   
    Y_classes = [np.argmax(item) for item in Y_test]
    #print(Y_classes)
    predicted_classes = [np.argmax(item) for item in predicted]
    
    predictions = np.array(predicted_classes)
    Y_actual = np.array(Y_classes)
   
    y_actu = pd.Series(np.reshape(Y_actual,len(Y_actual)), name='Actual')
    y_pred = pd.Series(np.reshape(predictions, len(predictions)), name='Predicted')
    
    y_act = pd.Categorical(y_actu, categories=[0,1,2,3,4])
    y_pre = pd.Categorical(y_pred, categories=[0,1,2,3,4])
    
    #F1 score
    from sklearn.metrics import f1_score
    print("Average F1 score is {}".format(f1_score(Y_classes, predicted_classes, average='weighted', labels = [0,1,2,3,4])))
    
    df_confusion = pd.crosstab(y_act, y_pre, rownames =['Actual'],colnames=['Predicted'])
    print("Confusion matrix: \n", df_confusion)
    #pprint.pprint(df_confusion)
    predicted= np.array(predicted).tolist()
    Y_test = np.array(Y_test).tolist()

    n_classes = 5
    print("n_classes is: ", n_classes)
    Y_test= np.asarray(Y_test)
    predicted = np.asarray(predicted)
    
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(Y_test[:, i], predicted[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(Y_test.ravel(), predicted.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    lw = 2
     #Compute macro-average ROC curve and ROC area
    # first aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Plot all ROC curves
    plt.figure()
    plt.plot(fpr["micro"], tpr["micro"],
             label='micro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["micro"]),
             color='deeppink', linestyle=':', linewidth=4)

    plt.plot(fpr["macro"], tpr["macro"],
             label='macro-average ROC curve (area = {0:0.2f})'
                   ''.format(roc_auc["macro"]),
             color='navy', linestyle=':', linewidth=4)

    colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                 label='ROC curve of class {0} (area = {1:0.2f})'
                 ''.format(i, roc_auc[i]))

    plt.plot([0, 1], [0, 1], 'k--', lw=lw)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc="lower right")
    plt.show()

In [98]:
#checkpoint save weights
#Y_train

hdf5FileName = "bestWeightsLSTMStateful3L50.hdf5"
checkpoint = ModelCheckpoint(hdf5FileName,monitor='val_acc',verbose=1,save_best_only=True,mode='max',save_weights_only=True)

In [112]:
class ResetStatesCallback(Callback):
    def __init__(self):
        self.counter = 0

    def on_batch_begin(self, batch, logs={}):
        if self.counter % (max_len*10) == 0:
            self.model.reset_states()
            print("Model reset. ",self.counter)
            #self.counter = 0
        self.counter += 1
        #print(self.counter)
        

In [ ]:
#Time to train

Batch_size=1
start_time = time.time()

x_train,y_train = processTrainTestArrays(X_train,Y_train)
x_val,y_val = processTrainTestArrays(X_val,Y_val)
model = create_model(x_train,y_train, 50)

#for i in range(100):
x = np.expand_dims(np.expand_dims(x_train.flatten(), axis=1), axis=1)
y = np.expand_dims(np.array([[v]*10 for v in y_train.flatten()]).flatten(), axis=1)

x_val = np.expand_dims(np.expand_dims(x_val.flatten(), axis=1), axis=1)
y_val = np.expand_dims(np.array([[v]*10 for v in y_val.flatten()]).flatten(), axis=1)
model.fit(x, y, epochs=15, batch_size=Batch_size, verbose=1, shuffle=False,
            validation_data = (x_val,y_val),callbacks=[checkpoint,ResetStatesCallback()])

#model.fit(x_train, y_train, epochs=15, batch_size=Batch_size, verbose=1, shuffle=False,
#            validation_data = (x_val,y_val),callbacks=[checkpoint])


print("--- %s seconds ---" % (time.time() - start_time))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (1, 50)                   10400     
_________________________________________________________________
dense_14 (Dense)             (1, 1)                    51        
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________
None
Train on 24880 samples, validate on 8300 samples
Epoch 1/15
Model reset.  0
   21/24880 [..............................] - ETA: 24:19 - loss: 0.6667 - acc: 0.4762      

/Users/ass6ash/anaconda/lib/python3.6/site-packages/keras/callbacks.py:93: UserWarning: Method on_batch_begin() is slow compared to the batch update (0.479706). Check your callbacks.
  % delta_t_median)


24880/24880 [==============================] - 196s 8ms/step - loss: 1.6592 - acc: 0.2363 - val_loss: 1.7229 - val_acc: 0.2205
Epoch 2/15
Model reset.  24880
24880/24880 [==============================] - 210s 8ms/step - loss: 1.6592 - acc: 0.2363 - val_loss: 1.7229 - val_acc: 0.2205
Epoch 3/15
Model reset.  49760
12308/24880 [=============>................] - ETA: 1:33 - loss: 1.6362 - acc: 0.2389     

In [114]:
#Test with best model
#Load weights and compile again
print("\n===========================\nTime for testing\n===========================\n")
model.load_weights(hdf5FileName)
nadam = optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
model.compile(loss='mean_squared_error', optimizer=nadam, metrics=['accuracy'])
print("Optimal weights loaded from file {}".format(hdf5FileName))
print("Model Successfully compiled with loaded weights\n")

#Do same preprocessing for test data
x_test,y_test = processTrainTestArrays(X_test,Y_test)
x_test = np.expand_dims(np.expand_dims(x_test.flatten(), axis=1), axis=1)
y_test = np.expand_dims(np.array([[v]*10 for v in y_test.flatten()]).flatten(), axis=1)

loss,acc = model.evaluate(x_test,y_test,batch_size=Batch_size)
print("Loss for testing = {} and Accuracy for testing = {}".format(loss,acc))
predicted = model.predict(x_test,batch_size=Batch_size)

#convert back to onehotencode
predicted = onehotencode(predicted)
y_test = onehotencode(y_test)
compute_metrics(predicted, y_test)    



Time for testing

Optimal weights loaded from file bestWeightsLSTMStateful3L50.hdf5
Model Successfully compiled with loaded weights

8300/8300 [==============================] - 21s 2ms/step

/Users/ass6ash/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


IndexError: index 2 is out of bounds for axis 1 with size 2

In [ ]:
predicted[0]

In [27]:
#temp =[np.argmax(item)+1 for item in predicted]

In [28]:
#temp1 = [np.argmax(item)+1 for item in Y_test]

In [72]:
len(X_train)

2488

In [110]:
x_temp[1]

array([[-0.1127386]])

In [107]:
x_temp = np.expand_dims(np.expand_dims(x_train.flatten(), axis=1), axis=1)